<a href="https://colab.research.google.com/github/GonzoDen/NMA-CN/blob/main/NMA_Porc_%26_Pines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Research Question:** What cortical regions are responsible for math and story problem solving? How do cortical regions interact to produce math and story problem processing?

Simplest model:
1. Maybe we can start by testing whether our program can extract the data
  - Copy necessary data from the initial data notebook
  - Extract most active 20+20 parcels
  - Build a matrix for ONE subject and 20 language parcels
      - No mean, I guess

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

Basic Parameters

In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339 ##initial 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360 ##

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
##N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
## initial BOLD_NAMES are bigger removed for the sake of project
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL", ##TODO: how to find resting state code
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR", ##TA: which direction we need to choose
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]

# You may want to limit the subjects used during code development.
# This will use all subjects:
subjects = range(N_SUBJECTS)

**DOWNLOADING DATA **

In [ ]:
#!rm -rf hcp
#maybe then creating a bash command to delete anything that is not related to the language task?

TODO: probably will have to delete folders later or whatever 

In [ ]:
fname = "hcp_task.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/s4h8j/download/
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

Only language part (still loading btw):
- will it distribute subjects in different folders?

Loading data:

In [ ]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
  ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids

def load_timeseries(subject, name, runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.
  
  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment 
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
      load_single_timeseries(subject, offset + run, remove_mean) for run in runs
  ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


def load_single_timeseries(subject, bold_run, remove_mean=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

def load_evs(subject, name, condition):
  """Load EV (explanatory variable) data for one task condition.

  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of task
    condition (str) : Name of condition

  Returns
    evs (list of dicts): A dictionary with the onset, duration, and amplitude
      of the condition for each run.

  """
  evs = []
  for id in get_image_ids(name):
    task_key = BOLD_NAMES[id - 1]
    ev_file = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{condition}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev = dict(zip(["onset", "duration", "amplitude"], ev_array))
    evs.append(ev)
  return evs

Task-based Analysis:

In [ ]:
def condition_frames(run_evs, skip=0):
  """Identify timepoints corresponding to a given condition in each run.

  Args:
    run_evs (list of dicts) : Onset and duration of the event, per run
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    frames_list (list of 1D arrays): Flat arrays of frame indices, per run

  """
  frames_list = []
  for ev in run_evs:

    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)

    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)

    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(skip, d) for s, d in zip(start, duration)]

    frames_list.append(np.concatenate(frames))

  return frames_list


def selective_average(timeseries_data, ev, skip=0):
  """Take the temporal mean across frames for a given condition.

  Args:
    timeseries_data (array or list of arrays): n_parcel x n_tp arrays
    ev (dict or list of dicts): Condition timing information
    skip (int) : Ignore this many frames at the start of each trial, to account
      for hemodynamic lag

  Returns:
    avg_data (1D array): Data averagted across selected image frames based
    on condition timing

  """
  # Ensure that we have lists of the same length
  if not isinstance(timeseries_data, list):
    timeseries_data = [timeseries_data]
  if not isinstance(ev, list):
    ev = [ev]
  if len(timeseries_data) != len(ev):
    raise ValueError("Length of `timeseries_data` and `ev` must match.")

  # Identify the indices of relevant frames
  frames = condition_frames(ev, skip)

  # Select the frames from each image
  selected_data = []
  for run_data, run_frames in zip(timeseries_data, frames):
    run_frames = run_frames[run_frames < run_data.shape[1]]
    selected_data.append(run_data[:, run_frames])

  # Take the average in each parcel
  avg_data = np.concatenate(selected_data, axis=-1).mean(axis=-1)

  return avg_data

In [ ]:
timeseries_task = []
for subject in subjects:
  timeseries_task.append(load_timeseries(subject, "language", concat=False))

Dividing into subsets:

In [ ]:
train_set = []
test_set = []

for subject in subjects:
    if np.random.choice(2, p=[0.6, 0.4])==0:
        train_set.append(subject) 
        
    else:
        test_set.append(subject)

In [ ]:
print(train_set)

[0, 2, 3, 5, 6, 11, 12, 16, 17, 18, 19, 20, 21, 22, 24, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 41, 44, 45, 46, 49, 50, 52, 55, 57, 59, 61, 64, 65, 66, 67, 68, 70, 71, 72, 73, 77, 78, 79, 80, 81, 83, 85, 87, 89, 90, 91, 92, 93, 94, 95, 97, 99, 100, 102, 103, 104, 109, 110, 111, 112, 113, 114, 119, 120, 124, 126, 128, 129, 132, 133, 134, 135, 136, 137, 138, 143, 145, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 163, 164, 165, 167, 168, 172, 173, 177, 178, 179, 181, 182, 183, 184, 185, 186, 188, 189, 192, 193, 197, 198, 199, 200, 201, 202, 203, 204, 207, 210, 211, 212, 215, 217, 218, 220, 221, 222, 224, 225, 226, 227, 229, 230, 233, 234, 235, 238, 239, 241, 242, 243, 244, 245, 249, 250, 253, 255, 256, 257, 258, 259, 263, 265, 267, 269, 270, 272, 273, 274, 276, 278, 280, 281, 282, 283, 288, 289, 291, 295, 299, 300, 304, 305, 306, 307, 309, 311, 313, 318, 319, 321, 322, 323, 324, 325, 326, 327, 329, 332, 334, 335, 338]


Substraction Analysis:

In [ ]:
task = "language"

N_MAX_PARCELS = 20

In [ ]:
#story
conditions_story = ["response_story","cue"]   # Run a substraction analysis between two conditions

contrast_story = []

for subject in subjects:

  # Get the average signal in each region for each condition
  evs = [load_evs(subject, task, cond) for cond in conditions_story] ##??? how can we get conditions
  avgs = [selective_average(timeseries_task[subject], ev) for ev in evs]

  # Store the region-wise difference
  contrast_story.append(avgs[0] - avgs[1])

group_contrast_story = np.mean(contrast_story, axis=0) #a contrast btw cond1 and 2 of the "average" subject 

In [ ]:
max_ind_story = group_contrast_story.argsort()[-N_MAX_PARCELS:][::-1] #found intexes of the most active parcels for story activity
#thresh = min(group_contrast_story[max_ind])

In [ ]:
#math
conditions_math = ["response_math","cue"]   # Run a substraction analysis between two conditions

contrast_math = []

for subject in subjects:

  # Get the average signal in each region for each condition
  evs = [load_evs(subject, task, cond) for cond in conditions_math] ##??? how can we get conditions
  avgs = [selective_average(timeseries_task[subject], ev) for ev in evs]

  # Store the region-wise difference
  contrast_math.append(avgs[0] - avgs[1])

group_contrast_math = np.mean(contrast_math, axis=0) #a contrast btw cond1 and 2 of the "average" subject 

In [ ]:
max_ind_math = group_contrast_math.argsort()[-N_MAX_PARCELS:][::-1] #found intexes of the most active parcels for math activity
#thresh = min(group_contrast_math[max_ind])

In [ ]:
print(max_ind_story)
print(max_ind_math)

[ 87 267 355 164 344 307  64 255 127 244 245 254 122 310 302 311 130 175
  65 340]
[259 229 225 208  49  83  45 143 274  28 323 325  79 324  30  95  94 296
 262 116]


Creating an input Matrix:
- does it work for both training and test data? 
    - Yes, I guess just when we will call function, we will send different args
    - Don't forget to divide dataset!!! (60/40) 

In [ ]:
def create_matrix(n_subjects, parcels_list, ): 
#filling matrix with values of

#filling matrix for test set

In [ ]:
#test function on different subsets